In [1]:
#install all dependencies
!pip3 install simple-tensor
!pip3 install opencv-python
!pip3 install comdutils
!pip3 uninstall tensorflow-gpu
!pip3 install tensorflow-gpu==1.15.0

     |████████████████████████████████| 51kB 2.2MB/s 
  Created wheel for simple-tensor: filename=simple_tensor-0.7.14-cp36-none-any.whl size=50840 sha256=9426d25d80d8260178bf5ec6cacf78b6b9289decf2b45031b4a61abbfe9238ea
  Stored in directory: /root/.cache/pip/wheels/c2/13/0c/be211db1c202df1c18498f5036a26449f182f6a096f093eb58
Successfully built simple-tensor
  Created wheel for comdutils: filename=comdutils-0.3.5-cp36-none-any.whl size=5222 sha256=f77538b1a3c5e3bacd40a227f0a0b6c355909404213d115b17d2b3455da28b6b
  Stored in directory: /root/.cache/pip/wheels/18/2b/82/f8681e425164b99fe7048ee4a72a7b63245c9a4375df2c5d2b
Successfully built comdutils
     |████████████████████████████████| 411.5MB 43kB/s 
     |████████████████████████████████| 51kB 6.0MB/s 
     |████████████████████████████████| 3.8MB 35.1MB/s 
     |████████████████████████████████| 512kB 38.3MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=2ce81f94e3044d09a78a7eec3b73453fb7ddcaa35e5bc

In [2]:
# connect to google drive
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
# import our dependencies
import tensorflow as tf
from simple_tensor.tensor_losses import softmax_crosentropy_mean
from simple_tensor.transfer_learning.image_recognition import *

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.




In [4]:
# ----------------------------------------- #
# Build simple tensor wrapper for image recognition
# there are 3 classes separated by folder "kelapa", "kurma", "lontar"
#
#
# folder structure:
# -dataset
# -    - train
# -    -     - kelapa
# -    -     - kurma
# -    -     - lontar
# -    - val
# -    -     - kelapa
# -    -     - kurma
# -    -     - lontar
#
#
# input image designed 300x300 in size
# with three channels            
# ----------------------------------------- #
imrec = ImageRecognition(classes=['kelapa', 'kurma', 'lontar'],
                         input_height = 300,
                         input_width = 300, 
                         input_channel = 3)

In [5]:
# ----------------------------------------- #
# build neural network graph
# we are using densenet 121 base architecture
# ----------------------------------------- #
out, var_list = imrec.build_densenet_base(imrec.input_placeholder,
                                    dropout_rate = 0.20,
                                    is_training = True,
                                    top_layer_depth = 128)

# ----------------------------------------- #
# calculate the loss
# softmax cross entropy loss
# update ops for TF batch normalization
# ----------------------------------------- #
cost = softmax_crosentropy_mean(out, imrec.output_placeholder)
update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
with tf.control_dependencies(update_ops):
    optimizer = tf.train.AdamOptimizer(learning_rate=0.0001).minimize(cost)


Instructions for updating:
Please use `layer.__call__` method instead.


Instructions for updating:
Use keras.layers.BatchNormalization instead.  In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.batch_normalization` documentation).
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [6]:
# ----------------------------------------- #
# Build TF saver for loading and saving model    
# Build TF session for running the operations
#
# saver_all : save/restore all variable in neural net
# saver_partial: save partial variable 
# ----------------------------------------- #
saver_partial = tf.train.Saver(var_list)
saver_all = tf.train.Saver()
session = tf.Session()
session.run(tf.global_variables_initializer())

# Note, we will use transfer learning
# So, for the first training we will load pretrained model partially
# why ....? cause we modify the architecture 
saver_partial.restore(sess=session, save_path='/content/gdrive/My Drive/model/densenet-121/tf-densenet121.ckpt')


# for the next training use this
# restore all previously trained model variable
#saver_partial.restore(sess=session, save_path='/content/gdrive/My Drive/tutorial/model')

INFO:tensorflow:Restoring parameters from /content/gdrive/My Drive/model/densenet-121/tf-densenet121.ckpt


In [9]:
# ----------------------------------------- #
# Create batch generator for train and val
# ----------------------------------------- #
train_generator = imrec.batch_generator(batch_size=24, 
                                        dataset_path='/content/gdrive/My Drive/dataset/intro-classification/train/', 
                                        message="TRAIN")
val_generator = imrec.batch_generator(batch_size=36, 
                                        dataset_path='/content/gdrive/My Drive/dataset/intro-classification/val/', 
                                        message="VAL")

In [ ]:
# ----------------------------------------- #
# start training
# iteration : num of iterartion
# subdivition: subdivition per iteration
# path_tosave_model: path to save model
# ----------------------------------------- #
imrec.optimize(iteration=2000, 
         subdivition=5,
         cost_tensor=cost,
         optimizer_tensor=optimizer,
         out_tensor = out,
         session = session, 
         saver = saver_all,
         train_generator = train_generator,
         val_generator = val_generator,
         best_loss = 10000,
         path_tosave_model='/content/gdrive/My Drive/tutorial/model')

------------------------INFO IMAGES-------------------
Image Folder: /content/gdrive/My Drive/dataset/intro-classification/train/
Number of Image in kelapa:  126
Number of Image in kurma:  128
Number of Image in lontar:  101
------------------------------------------------------
> Train sub 0 loss :  0.97123957
> Train sub 1 loss :  0.9582165
> Train sub 2 loss :  1.0214163
> Train sub 3 loss :  0.969234
> Train sub 4 loss :  0.99071336
------------------------INFO IMAGES-------------------
Image Folder: /content/gdrive/My Drive/dataset/intro-classification/val/
Number of Image in kelapa:  12
Number of Image in kurma:  12
Number of Image in lontar:  13
------------------------------------------------------
>> epoch:  0 train loss:  0.982 val loss:  1.007 val acc:  0.639 ************* model saved
> Train sub 0 loss :  0.9828984
> Train sub 1 loss :  0.96007895
> Train sub 2 loss :  0.956199
> Train sub 3 loss :  0.9587922
> Train sub 4 loss :  0.950988
==>>> INFO: your VAL in class kela